In [ ]:

import numpy as np
import pandas as pd
from   scipy.stats import gamma as gamma_dist
import matplotlib.pyplot as plt
import bayesflow as bf
import tensorflow as tf

# CONSTANTS
ALPHA      = 9                # Gamma shape (From Original Artical)
ETA_CONST  = 1e-3             # saliency noise – From Original Artical)
PATH       = "fixseqin_PB2expVP10.dat"   #data file

# 0)  LOAD FIXATIONS   
def load_fixations(path=PATH):
    cols = ['sentID','fixNum','onset','dur_ms','wordID',
            'x','y','line','flag1','flag2']
    return (
        pd.read_csv(path, sep=r'\s+', header=None, names=cols,
                    usecols=['sentID','wordID','dur_ms'])
          .query("dur_ms >= 50")          # remove microsaccades, For "cognative Fixations It is important"
          .reset_index(drop=True)
    )

df           = load_fixations()
sent_lens    = df.groupby("sentID")["wordID"].max().to_dict()   # {id: N_words}
all_sent_ids = np.array(list(sent_lens), dtype=int)
GLOBAL_PAD   = max(sent_lens.values())

# 1)  SIMULATOR  (9 raw channels, uses fixed η)
def gamma_timer(mu_T):
    """Draw a single fixation duration ∼ Gamma(α, scale=μ_T/α)."""
    return np.random.gamma(ALPHA, mu_T / ALPHA)

def swift_baseline(nu, r, mu_T, *, sent_ids, pad_len):  
    """
    Produce one batch of padded scan-paths with **fixed** η.
    Shapes of all channels: (B, T, 1)  — no extra dummy axes.
    """
    sent_ids = np.atleast_1d(sent_ids)          # (B,) even if B=1
    pad_len  = np.atleast_1d(pad_len)           # same
    B        = sent_ids.shape[0]
    T        = int(pad_len[0])
    zeros = lambda: np.zeros((B, T, 1), np.float32)

    fd, fw = zeros(), zeros()      # fix_dur, fix_word
    sl, sd = zeros(), zeros()      # sacc_len, sacc_dir
    le     = zeros()               # landing_error
    skp, rf= zeros(), zeros()      # skip_flag, refix_flag

    for b, sid in enumerate(sent_ids):
        N = sent_lens[int(sid)]
        for w in range(N):
            fd[b, w, 0] = gamma_timer(mu_T[b])
            fw[b, w, 0] = w + 1
            L           = np.random.poisson(r[b])
            sl[b, w, 0] = L
            sd[b, w, 0] = 1.0                      # always forward
            le[b, w, 0] = np.random.normal(0., 5. * ETA_CONST)

            p = 1 / (1 + np.exp(-20 * (nu[b] - 0.5)))
            skipped = np.random.rand() < p
            skp[b, w, 0] = skipped
            if not skipped:
                rf[b, w, 0] = np.random.rand() < p

    # sentence-level summary channels, copied along T
    skip_rate  = np.repeat(skp.mean(axis=1, keepdims=True), T, axis=1)
    refix_rate = np.repeat(rf .mean(axis=1, keepdims=True), T, axis=1)

    return dict(fix_dur=fd, fix_word=fw, sacc_len=sl, sacc_dir=sd,
                landing_error=le, skip_flag=skp, refix_flag=rf,
                skip_rate=skip_rate, refix_rate=refix_rate)

# 2)  PRIOR, META,  SIMULATOR WRAPPER
def sample_prior(batch_shape=None, **_):
    B = int(np.prod(batch_shape)) if batch_shape else 1
    return dict(
        nu   = np.random.uniform(0, 1 , B),
        r    = np.random.uniform(0 , 12 , B),
        mu_T = np.random.uniform(100, 400, B)
    )

def batch_meta(batch_shape):
    B    = int(batch_shape[0])
    sids = np.random.choice(all_sent_ids, size=B)
    return dict(sent_ids=sids.astype(np.int32),
                pad_len=np.full(B, GLOBAL_PAD, np.int32))

def simulator_fn(nu, r, mu_T, *, sent_ids, pad_len):
    """Wrapper that feeds fixed η to the baseline generator."""
    return swift_baseline(nu, r, mu_T,
                          sent_ids = sent_ids,
                          pad_len  = pad_len)

simulator = bf.make_simulator(
    [sample_prior, simulator_fn],  # building blocks
    {},                             # no object kwargs
    batch_meta
)

# 3)  NETWORKS  +  ADAPTER
inf_net = bf.networks.FlowMatching()
sum_net = bf.networks.SetTransformer(
    embed_dims =(64,), num_heads =(4,), mlp_depths=(2,), mlp_widths=(128,)
)

adapter = (
    bf.Adapter()
      .concatenate(["nu", "r", "mu_T"], into="inference_variables")

      # (B,1,T,1) → (B,T,1)
      .squeeze(
          ["fix_dur","fix_word","sacc_len","sacc_dir","landing_error",
           "skip_flag","refix_flag","skip_rate","refix_rate"],
          axis=1)

      #  9 channel → summary_variables  (B,T,9)
      .concatenate(
          ["fix_dur","fix_word","sacc_len","sacc_dir","landing_error",
           "skip_flag","refix_flag","skip_rate","refix_rate"],
          into="summary_variables")
)


# 4)  WORKFLOW
workflow = bf.BasicWorkflow(
    simulator        = simulator,
    adapter          = adapter,
    inference_network= inf_net,
    summary_network  = sum_net,
    inference_variables = ["nu", "r", "mu_T"],

)
print("Workflow initialised, ready to train.")

# For Debug check:   
raw_test = simulator.sample((4,))        
print("\nMini-batch channel shapes:")
for k, v in raw_test.items():
    print(f"{k:12s} {v.shape}")

# 5)  Training Part
history = workflow.fit_online(
    epochs                = 50,
    num_batches_per_epoch = 200,
    batch_size            = 64,
    validation_data       = 200,     # → every epoch: 200 fresh sims
    loss                  = "mmd",
    optimizer             = tf.keras.optimizers.Adam(1e-3),
    verbose               = 2,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor="val_loss",
            patience=6,          
            min_delta=0.0,      
            restore_best_weights=True
        ),
        ],
)


print("\nFirst 5 losses:", history.history["loss"][:5])


In [ ]:
import bayesflow.diagnostics as bfd
import matplotlib.pyplot as plt
import numpy as np

S = 200                     
raw_test = workflow.simulate(batch_shape=(S,))   # simulator → raw dict
test_data = workflow.adapter(raw_test)           


M = 1000
posterior = workflow.sample(
    num_samples = M,
    conditions  = test_data      
)



figs = workflow.plot_default_diagnostics(
    test_data   = raw_test,   
    num_samples = M
)

for name, fig in figs.items():  
    print(name)
    fig.show()



In [ ]:
def pack_scans_to_raw(scans, pad_len):
    """
    Nine channels, each shape (B, 1, pad_len, 1)  ← cümle ekseni = 1
    """
    B, T = len(scans), pad_len
    zeros = lambda: np.zeros((B, 1, T, 1), np.float32)

    raw = dict(fix_dur=zeros(),  fix_word=zeros(),
               sacc_len=zeros(), sacc_dir=zeros(),
               landing_error=zeros(),
               skip_flag=zeros(), refix_flag=zeros(),
               skip_rate=zeros(),  refix_rate=zeros())

    for b, sc in enumerate(scans):
        n = len(sc["fix_dur"])
        raw["fix_dur"][b, 0, :n, 0]   = sc["fix_dur"]
        raw["fix_word"][b,0, :n, 0]   = sc["fix_word"]
        raw["sacc_len"][b,0, :n, 0]   = sc["sacc_len"]
        raw["sacc_dir"][b,0, :n, 0]   = np.sign(sc["sacc_len"])
        raw["skip_flag"][b,0,:n,0]    = sc["skip_flag"]
        raw["refix_flag"][b,0,:n,0]   = sc["refix_flag"]

        raw["skip_rate"][b,0,:,0]   = sc["skip_flag"].mean()
        raw["refix_rate"][b,0,:,0]  = sc["refix_flag"].mean()
    return raw

def build_scanpaths(df):
    scans=[]
    for sid,g in df.groupby('sentID',sort=True):
        g = g.sort_values('fixNum') if 'fixNum' in g else g.sort_index()
        w_ids = g.wordID.to_numpy()
        durs  = g.dur_ms.to_numpy(float)
        sacc  = np.diff(np.r_[w_ids[0], w_ids])
        skip  = (np.abs(sacc) > 1).astype(int)
        refix = np.r_[0,(w_ids[1:]==w_ids[:-1]).astype(int)]
        scans.append(dict(
            fix_word   = w_ids.astype(int),
            fix_dur    = durs,
            sacc_len   = sacc.astype(int),
            skip_flag  = skip,
            refix_flag = refix,
        ))
    return scans



# ---------------- REAL PARTICIPANT INFERENCE ------------------
df_real   = load_fixations(PATH)
DATA_REAL = build_scanpaths(df_real)

real_raw  = pack_scans_to_raw(DATA_REAL, GLOBAL_PAD)
# add dummy θ keys so that adapter finds them
real_raw["nu"]   = np.array([[0.]], np.float32)
real_raw["r"]    = np.array([[0.]], np.float32)
real_raw["mu_T"] = np.array([[0.]], np.float32)

cond      = workflow.adapter(real_raw, stage="predict")

posterior = workflow.sample(num_samples=1000, conditions=cond)

print("\nPosterior means (reader-specific):")
for p, v in posterior.items():
    print(f"{p:4s}: μ={v.mean():7.3f}  σ={v.std():6.3f}")

plt.figure(figsize=(8,2.5))
for i,(p,v) in enumerate(posterior.items(),1):
    plt.subplot(1,len(posterior),i)
    plt.hist(v.flatten(), 40, density=True, alpha=.6)
    plt.axvline(v.mean(), c='k', ls='--'); plt.title(p)
plt.tight_layout(); plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Choose the first subject (index 0)
subject_idx = 0

# Extract and reshape to (num_samples, num_variables)
samples = np.column_stack([
    posterior["nu"][subject_idx].flatten(),
    posterior["r"][subject_idx].flatten(),
    posterior["mu_T"][subject_idx].flatten()
])

# Convert to dict as expected by BayesFlow
posterior_dict_single = {
    "nu": samples[:, 0][:, None],
    "r": samples[:, 1][:, None],
    "mu_T": samples[:, 2][:, None]
}

grid = bf.diagnostics.plots.pairs_posterior(
    estimates=posterior_dict_single,
    variable_keys=["nu", "r", "mu_T"],
    height=2.3,
    alpha=0.35
)

grid.map_diag(sns.histplot, bins=40, color="#1f77b4", edgecolor="w")
plt.show()


In [ ]:
# For Debug Check Posterior Values.


for k, v in posterior_dict.items():
    print(k, v.mean(), v.std(), v.min(), v.max())


nu 0.8789034 0.16725872 0.36488742 1.3749603
r 1.5622747 0.814267 -1.3081803 6.244174
mu_T 123.00348 19.42611 36.833023 218.0245


In [8]:
import numpy
import pandas
import scipy
import matplotlib
import seaborn
import tensorflow
import bayesflow

print("numpy:", numpy.__version__)
print("pandas:", pandas.__version__)
print("scipy:", scipy.__version__)
print("matplotlib:", matplotlib.__version__)
print("seaborn:", seaborn.__version__)
print("tensorflow:", tensorflow.__version__)
print("bayesflow:", bayesflow.__version__)


numpy: 1.26.4
pandas: 2.2.3
scipy: 1.15.2
matplotlib: 3.10.1
seaborn: 0.13.2
tensorflow: 2.19.0
bayesflow: 2.0.4
